In [2]:
import requests

events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1337055, "item_id": 17245}

# resp = requests.post(events_store_url + "/put", headers=headers, params=params)
# if resp.status_code == 200:
#     result = resp.json()
# else:
#     result = None
#     print(f"status code: {resp.status_code}")
    
# print(result) 

resp = requests.post(events_store_url + "/get", 
                     headers=headers, 
                     params={"user_id": 1127794, "k": 3})
print(resp.json()) 

{'events': [4731479, 7785, 18734992, 18734992]}


In [4]:
params = {"user_id": 1291248, 'k': 3}
recommendations_url = "http://127.0.0.1:8020"
resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs) 

KeyboardInterrupt: 